In [1]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm0104",
    security_token="EwVPSL02qXM8XQ2pkUdA13aW",
)

In [2]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
    ]
]
Empresas_SF.shape

(7924, 7)

In [3]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(7928, 5)

In [4]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A"
    ]
]

Programas_empresas.shape

(7932, 9)

fin empresas y comienzo de rescate de pacientes


In [5]:
#CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        'Phone'
        
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.

In [ ]:
#buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña"        
    }
)
Paciente_Programas_SF.shape

In [ ]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c",
)
empresas_unidas.shape

In [ ]:
nuevos = ("F:/Empresas SF/reportes/carga.xlsx")
campaña = 'METLIFE'
Pais = 'Chile'




Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].str.replace("-", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()                                       

In [ ]:
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
#Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(
    "-", ""
)


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(
    Pacientes_SF["Rut"])

In [ ]:
#aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

In [ ]:
Pacientes_con_id.head()

In [ ]:
#Pacientes_con_id.to_excel("F:/Empresas SF/reportes/carga2.xlsx",index=False)

In [ ]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = Pacientes_con_id["rut"]
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = (
        Pacientes_con_id["email"].astype(str).str.lower().replace("nan", "")
    )
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]
if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = Pacientes_con_id["celular"]

Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"
Subir_pacientes["BillingCountry"] = "Chile"
Subir_pacientes["Identification_Type__c"] = "RUT"
Subir_pacientes["RutEstaoNo"] = Pacientes_Archivo["RutEstaoNo"]

if Pais == 'Chile':    
    Subir_pacientes['Pa_s_del_Paciente__c'] = 'a1U8c000005dAtnEAE'
else:
    Subir_pacientes['Pa_s_del_Paciente__c'] = 'a1U8c000004cvS7EAI'


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

In [ ]:
    
clinica_virtual = pd.DataFrame()    
clinica_virtual['rut'] = Pacientes_con_id["rut1"].str.upper()
clinica_virtual['firstName'] = Pacientes_con_id['nombres']
clinica_virtual['lastName'] = Pacientes_con_id['apellidos']
clinica_virtual['accountId'] = campaña



Codigos_Clinica_Virtual = {
'BCI':'1',
'Farmacias Ahumada':'2',
'MetLife':'3',
'Vida Security':'4',
'Willis':'5',
'Entel':'6',
'Testing':'7',
'3SP':'8',
'HomeCare':'9',
'Pedro de Valdivia':'10',
'Colaboradores':'11',
'MineraAbra':'12',
'CMPC':'13',
'Lipigas':'14',
'Asismed':'15',
'Siempre Perú':'16',
'Farmacias Cruz Verde':'17',
'WFS Food Services':'18',
'VidaCel':'19',
'Dermik':'20',
'EY':'21',
'IND':'22',
'Sindicato Walmart':'23',
'Instruvalve':'24',
'VidaCel Colaboradores':'25',
'Socofar':'26',
'Milan Fajanovic':'27',
'THBseguros':'28',
'PESCO':'29',
'Burgos - Consorcio':'30',
'COFAMA-SILENTIUM':'31',
'KAUFMANN':'32',
'QuePlan':'33',
'Zurich':'34',
'Colaboradores Siempre Chile':'35',
'TALO':'36',
'TRANSVIP':'37',
'Vida Security Colaboradores':'38',
'FORUS':'39',
'SURA':'40'
}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)